In [1]:
import pickle
import numpy as np
import pandas as pd
import plotly.express as px

from pvd_io import *

In [2]:
data_dir = 'pvd_analysis/'
min_file_size = 1e3  # 1 kb
datasets, sessions, files = scan_directories(data_dir, min_file_size, filetype='.pkl')
print(f"Located {len(files)} matched segment lists.")

Located 95 matched segment lists.


In [3]:
# Load Pickled lists of change matrices
matched_segments_voxels = []
matched_segments_spline = []

for ii, session in enumerate(sessions):
    file_path_vx = f"{data_dir}/{datasets[ii]}/{session}/length_change_voxels.pkl"
    file_path_sp = f"{data_dir}/{datasets[ii]}/{session}/length_change_spline.pkl"
    with open(file_path_vx, 'rb') as file:
        session_segments_vx = pickle.load(file)
    with open(file_path_sp, 'rb') as file:
        session_segments_sp = pickle.load(file)
    matched_segments_voxels.append(session_segments_vx)
    matched_segments_spline.append(session_segments_sp)

In [4]:
len(matched_segments_voxels) == len(sessions)

True

In [21]:
# Take a given session and extract pairwise changes to a pandas df
seg_changes = []

for segment in matched_segments_voxels[2]:
    for ii in range(segment.shape[0]-1):
        change = segment[ii+1, ii]
        seg_changes.append(change)

seg_change_df = pd.DataFrame(seg_changes)
seg_change_df = seg_change_df.rename(columns={seg_change_df.columns[0]: 'changes'})
seg_change_df.head()


,changes
0,11
1,-1
2,2
3,4
4,-7


In [22]:
fig = px.histogram(seg_change_df, x='changes', nbins=30, histnorm='probability density')

# Calculate the mean of the data
mean_value = np.mean(seg_change_df)

# Add a vertical line for the mean
fig.add_vline(x=mean_value, 
              line_width=2, 
              line_dash="dash", 
              line_color="black", 
              annotation_text=f"{mean_value:.2f}", 
              annotation_position="top")

fig.show()

In [23]:
# Take all sessions and extract pairwise changes to a pandas df
seg_changes = []

for matched_segments in matched_segments_voxels:
    for segment in matched_segments:
        for ii in range(segment.shape[0]-1):
            change = segment[ii+1, ii]
            seg_changes.append(change)

    seg_change_df = pd.DataFrame(seg_changes)
    seg_change_df = seg_change_df.rename(columns={seg_change_df.columns[0]: 'changes'})
    seg_change_df.head()

In [25]:
fig = px.histogram(seg_change_df, x='changes', nbins=30, histnorm='probability density')

# Calculate the mean of the data
mean_value = np.mean(seg_change_df)

# Add a vertical line for the mean
fig.add_vline(x=mean_value, 
              line_width=1, 
              line_dash="dash", 
              line_color="black", 
              annotation_text=f"{mean_value:.2f}", 
              annotation_position="top")

fig.show()